<a href="https://colab.research.google.com/github/Rohianon/DataminingPDF/blob/main/PDFCatalog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fitz
!pip install PyMuPDF

In [ ]:
import requests
import fitz
from bs4 import BeautifulSoup
import re
import pandas as pd
pd.set_option('display.max_columns', 500)

In [ ]:
print(fitz.__doc__)


PyMuPDF 1.18.13: Python bindings for the MuPDF 1.18.0 library.
Version date: 2021-05-05 06:32:22.
Built for Python 3.7 on linux (64-bit).



In [ ]:
catalog = fitz.open('/content/2019-2020-Undergraduate-Catalog-web.pdf')

In [ ]:
catalog.page_count

282

In [ ]:
catalog.metadata

{'author': '',
 'creationDate': "D:20190821133731-04'00'",
 'creator': 'Adobe InDesign 14.0 (Macintosh)',
 'encryption': None,
 'format': 'PDF 1.7',
 'keywords': '',
 'modDate': "D:20190821161830-04'00'",
 'producer': 'Adobe PDF Library 15.0',
 'subject': '',
 'title': '',
 'trapped': ''}

In [ ]:
# get the table of contents
catalog.get_toc()

[]

In [ ]:
catalog

Document('/content/2019-2020-Undergraduate-Catalog-web.pdf')

In [ ]:
# read a random page
import random
pno = random.randint(0,catalog.page_count)
catalog.load_page(pno)

page 178 of /content/2019-2020-Undergraduate-Catalog-web.pdf

In [ ]:
page = catalog[pno]

In [ ]:
def findCourses(page):
    # load the page number input by user
    ## catalog.load_page(page) same as below
    page = catalog[page]
    # extract text as html content
    pg_content = page.get_text('html')
    # parse through beutiful soup using html.parser
    soup = BeautifulSoup(pg_content, 'html.parser')
    # use regex to find the partern of the courses
    course = re.findall('([A-Z.]+ [0-9.]+: [A-Za-z A-Za-z A-Za-z].+)',soup.get_text())
    # course = [i.split(':') for i in course]
    return course

In [ ]:
findCourses(177)

['ACCT 101:  Principles of Accounting I: Financial Accounting',
 'ACCT 102:  Principles of Accounting II: Managerial Accounting',
 'ACCT 140: Computer Applications in Accounting*',
 'ACCT 201:  Intermediate Accounting I',
 'ACCT 202:  Intermediate Accounting II',
 'ACCT 230:  Tax Accounting',
 'ACCT 335:  Corporate Tax Accounting',
 'ACCT 340:  Cost Accounting',
 'ACCT 350: Accounting for Government and Non-Profit Entities ',
 'ACCT 401:  Advanced Accounting',
 'ACCT 410:  Auditing',
 'ACCT 475: Accounting Internship',
 'ANTH 107: Language and Culture: Introduction to Anthropological ',
 'ANTH 112:  World Cultures',
 'ANTH 120:  Physical Anthropology']

In [ ]:
data = [findCourses(i) for i in range(170, catalog.page_count) if len(findCourses(i)) !=0]

In [ ]:
flat_data = [item for sublist in data for item in sublist]
flat_data

['ACCT 101:  Principles of Accounting I: Financial Accounting',
 'ACCT 102:  Principles of Accounting II: Managerial Accounting',
 'ACCT 140: Computer Applications in Accounting*',
 'ACCT 201:  Intermediate Accounting I',
 'ACCT 202:  Intermediate Accounting II',
 'ACCT 230:  Tax Accounting',
 'ACCT 335:  Corporate Tax Accounting',
 'ACCT 340:  Cost Accounting',
 'ACCT 350: Accounting for Government and Non-Profit Entities ',
 'ACCT 401:  Advanced Accounting',
 'ACCT 410:  Auditing',
 'ACCT 475: Accounting Internship',
 'ANTH 107: Language and Culture: Introduction to Anthropological ',
 'ANTH 112:  World Cultures',
 'ANTH 120:  Physical Anthropology',
 'ANTH 121:  Physical Anthropology Lab',
 'ANTH 124: Stories through Things: Anthropology and Material Culture',
 'ANTH 125: Introduction to Gender Studies',
 'ANTH 130:  Archaeology',
 'ANTH 131:  Archaeology Lab',
 'ANTH 200:  Prehistory of Eastern North America',
 'ANTH 201:  Prehistory of  Western North America',
 'ANTH 202:  Indians

In [ ]:
pattern = lambda x: re.match('[A-Z]+',x)
pattern1 = lambda x: re.match('[0-9]+',x)

In [ ]:
# pattern(data[0][2])
split1 = lambda x: x.split(': ',expand=True)




df = pd.DataFrame(flat_data, columns=['courses'])
df.head()

,courses
0,ACCT 101: Principles of Accounting I: Financi...
1,ACCT 102: Principles of Accounting II: Manage...
2,ACCT 140: Computer Applications in Accounting*
3,ACCT 201: Intermediate Accounting I
4,ACCT 202: Intermediate Accounting II


# 

# Data manipulation using pandas


In [ ]:
df[['c','title']]=df.courses.str.split(': ',n=1,expand=True,)
df.drop(columns=['courses'], inplace=True)
df.head()

,c,title
0,ACCT 101,Principles of Accounting I: Financial Accounting
1,ACCT 102,Principles of Accounting II: Managerial Accou...
2,ACCT 140,Computer Applications in Accounting*
3,ACCT 201,Intermediate Accounting I
4,ACCT 202,Intermediate Accounting II


In [ ]:
df[['prefix','number']]=df.c.str.split(' ',n=1,expand=True,)
df.drop(columns=['c'], inplace=True)
df.head()

,title,prefix,number
0,Principles of Accounting I: Financial Accounting,ACCT,101
1,Principles of Accounting II: Managerial Accou...,ACCT,102
2,Computer Applications in Accounting*,ACCT,140
3,Intermediate Accounting I,ACCT,201
4,Intermediate Accounting II,ACCT,202


In [ ]:
# Reoder the columns
df = df[['prefix','number','title']]

In [ ]:
# Final Data
df

,prefix,number,title
0,ACCT,101,Principles of Accounting I: Financial Accounting
1,ACCT,102,Principles of Accounting II: Managerial Accou...
2,ACCT,140,Computer Applications in Accounting*
3,ACCT,201,Intermediate Accounting I
4,ACCT,202,Intermediate Accounting II
...,...,...,...
1331,WL,312,Second Language Acquisition
1332,WL,314,Introduction to Sociolinguistics
1333,WL,501,Culturally and Linguistically Diverse Learners
1334,WL,512,Second-Language Acquisition


In [ ]:
df.to_csv('courses.csv',index=0)